# 🧬 Mutation Conformation Pipeline

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stdmedoth/annexin_a11_mutation_pipeline/blob/main/mutation_pipeline_colab.ipynb)

This notebook generates and analyzes protein conformations for mutations using BioEmu.

## Workflow:
1. Input wild-type (WT) protein sequence
2. Specify mutation to study
3. Generate conformational ensembles with BioEmu
4. Align and analyze conformations
5. Compare WT vs Mutant profiles

---

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install -q biopython mdtraj prody matplotlib seaborn plotly ipywidgets scipy

# Install BioEmu from Microsoft
# Option 1: From PyPI (if available)
!pip install -q bioemu 2>/dev/null || echo "BioEmu not on PyPI, trying GitHub..."

# Option 2: From GitHub
!pip install -q git+https://github.com/microsoft/bioemu.git 2>/dev/null || echo "Could not install from GitHub"

# Check if BioEmu is installed
try:
    import bioemu
    print("✓ BioEmu installed successfully!")
    BIOEMU_AVAILABLE = True
except ImportError:
    print("⚠ BioEmu not installed. Will use simulated data for demo.")
    print("  To install manually, visit: https://github.com/microsoft/bioemu")
    BIOEMU_AVAILABLE = False

print("✓ Other packages installed successfully!")

### 1.1 Google Drive Persistence (Colab)

Mount Google Drive to save/load results across sessions.

In [ ]:
# Google Colab Persistence Setup
import os

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("⚠ Not running in Google Colab - using local storage")

# Mount Google Drive
DRIVE_MOUNTED = False
DRIVE_OUTPUT_DIR = None

if IN_COLAB:
    from google.colab import drive
    
    try:
        drive.mount('/content/drive')
        DRIVE_MOUNTED = True
        print("✓ Google Drive mounted successfully!")
        
        # Create persistent output directory in Drive
        DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/MutationPipeline"
        os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)
        print(f"✓ Persistent storage: {DRIVE_OUTPUT_DIR}")
        
        # List existing results if any
        existing_results = os.listdir(DRIVE_OUTPUT_DIR) if os.path.exists(DRIVE_OUTPUT_DIR) else []
        if existing_results:
            print(f"\n📁 Found {len(existing_results)} existing result(s):")
            for item in existing_results[:10]:  # Show first 10
                print(f"   • {item}")
            if len(existing_results) > 10:
                print(f"   ... and {len(existing_results) - 10} more")
        else:
            print("\n📁 No previous results found")
            
    except Exception as e:
        print(f"⚠ Could not mount Google Drive: {e}")
        print("  Results will be saved locally (not persistent)")
else:
    print("  Results will be saved to local directory")

In [ ]:
# Persistence Helper Functions
import shutil
import json
from pathlib import Path

class PersistenceManager:
    """Manages saving and loading results to/from Google Drive."""
    
    def __init__(self, drive_dir: str = None, local_dir: str = "./mutation_pipeline_output"):
        self.drive_dir = Path(drive_dir) if drive_dir else None
        self.local_dir = Path(local_dir)
        self.local_dir.mkdir(exist_ok=True)
        
        # Use Drive if available, otherwise local
        self.output_dir = self.drive_dir if self.drive_dir and self.drive_dir.exists() else self.local_dir
        print(f"📂 Output directory: {self.output_dir}")
    
    def get_output_path(self, protein_name: str, mutation: str) -> Path:
        """Get the output path for a specific mutation analysis."""
        path = self.output_dir / protein_name / mutation
        path.mkdir(parents=True, exist_ok=True)
        return path
    
    def save_checkpoint(self, data: dict, name: str, output_path: Path) -> str:
        """Save a checkpoint file."""
        checkpoint_file = output_path / f"{name}_checkpoint.json"
        with open(checkpoint_file, 'w') as f:
            json.dump(data, f, indent=2, default=str)
        print(f"  💾 Checkpoint saved: {checkpoint_file.name}")
        return str(checkpoint_file)
    
    def load_checkpoint(self, name: str, output_path: Path) -> dict:
        """Load a checkpoint file if it exists."""
        checkpoint_file = output_path / f"{name}_checkpoint.json"
        if checkpoint_file.exists():
            with open(checkpoint_file, 'r') as f:
                data = json.load(f)
            print(f"  📥 Checkpoint loaded: {checkpoint_file.name}")
            return data
        return None
    
    def save_numpy_data(self, arrays: dict, output_path: Path) -> None:
        """Save numpy arrays for later analysis."""
        import numpy as np
        npz_file = output_path / "trajectory_data.npz"
        np.savez_compressed(npz_file, **arrays)
        print(f"  💾 Trajectory data saved: {npz_file.name}")
    
    def load_numpy_data(self, output_path: Path) -> dict:
        """Load numpy arrays if they exist."""
        import numpy as np
        npz_file = output_path / "trajectory_data.npz"
        if npz_file.exists():
            data = dict(np.load(npz_file))
            print(f"  📥 Trajectory data loaded: {npz_file.name}")
            return data
        return None
    
    def list_previous_analyses(self) -> list:
        """List all previous mutation analyses."""
        analyses = []
        if self.output_dir.exists():
            for protein_dir in self.output_dir.iterdir():
                if protein_dir.is_dir():
                    for mutation_dir in protein_dir.iterdir():
                        if mutation_dir.is_dir():
                            summary_file = mutation_dir / "analysis_summary.json"
                            if summary_file.exists():
                                with open(summary_file) as f:
                                    summary = json.load(f)
                                analyses.append({
                                    "protein": protein_dir.name,
                                    "mutation": mutation_dir.name,
                                    "path": str(mutation_dir),
                                    "timestamp": summary.get("timestamp", "unknown")
                                })
        return analyses
    
    def sync_to_drive(self, local_path: Path) -> None:
        """Sync local results to Google Drive."""
        if self.drive_dir and self.drive_dir != self.local_dir:
            dest = self.drive_dir / local_path.relative_to(self.local_dir)
            if local_path.is_dir():
                shutil.copytree(local_path, dest, dirs_exist_ok=True)
            else:
                dest.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(local_path, dest)
            print(f"  ☁️ Synced to Drive: {dest}")

# Initialize persistence manager
persistence = PersistenceManager(
    drive_dir=DRIVE_OUTPUT_DIR if DRIVE_MOUNTED else None,
    local_dir="./mutation_pipeline_output"
)

# Show previous analyses
previous = persistence.list_previous_analyses()
if previous:
    print(f"\n📋 Previous analyses found ({len(previous)}):")
    for analysis in previous[-5:]:  # Show last 5
        print(f"   • {analysis['protein']} - {analysis['mutation']} ({analysis['timestamp'][:10]})")
else:
    print("\n📋 No previous analyses found")

print("\n✓ PersistenceManager initialized")

In [ ]:
# Import libraries
import os
import re
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict, Any
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")

## 2. Core Modules

### 2.1 Sequence Handler

In [ ]:
# Valid amino acid codes
VALID_AMINO_ACIDS = set("ACDEFGHIKLMNPQRSTVWY")

class SequenceHandler:
    """Handles protein sequence operations."""
    
    def __init__(self, sequence: str, name: str = "protein"):
        self.raw_sequence = sequence
        self.sequence = self._clean_sequence(sequence)
        self.name = self._sanitize_name(name)
        self._validate()
    
    def _sanitize_name(self, name: str) -> str:
        """Sanitize protein name for filesystem compatibility."""
        # Replace problematic characters with underscores
        sanitized = re.sub(r'[|/\\:*?"<>]', '_', name)
        sanitized = sanitized.strip().strip('_')
        sanitized = re.sub(r'_+', '_', sanitized)
        return sanitized if sanitized else "protein"
    
    def _clean_sequence(self, sequence: str) -> str:
        """Remove whitespace and convert to uppercase."""
        return re.sub(r'\s+', '', sequence.upper())
    
    def _validate(self) -> None:
        """Validate the sequence contains only valid amino acids."""
        invalid_chars = set(self.sequence) - VALID_AMINO_ACIDS
        if invalid_chars:
            raise ValueError(
                f"Invalid amino acid(s) found: {', '.join(invalid_chars)}\n"
                f"Valid amino acids are: {''.join(sorted(VALID_AMINO_ACIDS))}"
            )
        if len(self.sequence) == 0:
            raise ValueError("Sequence cannot be empty")
    
    def __len__(self) -> int:
        return len(self.sequence)
    
    def __str__(self) -> str:
        return self.sequence
    
    def get_residue(self, position: int) -> str:
        """Get residue at a specific position (1-indexed)."""
        if position < 1 or position > len(self.sequence):
            raise ValueError(f"Position {position} out of range. Valid: 1-{len(self.sequence)}")
        return self.sequence[position - 1]
    
    def to_fasta(self) -> str:
        """Convert sequence to FASTA format."""
        lines = [f">{self.name}"]
        for i in range(0, len(self.sequence), 60):
            lines.append(self.sequence[i:i + 60])
        return "\n".join(lines)
    
    def get_stats(self) -> dict:
        """Calculate sequence statistics."""
        seq = self.sequence
        return {
            "length": len(seq),
            "charged": sum(seq.count(aa) for aa in "DEKRH"),
            "hydrophobic": sum(seq.count(aa) for aa in "AVILMFYW"),
            "polar": sum(seq.count(aa) for aa in "STNQ"),
        }
    
    @classmethod
    def from_fasta_string(cls, fasta_content: str) -> "SequenceHandler":
        """Parse FASTA content string."""
        lines = fasta_content.strip().split('\n')
        name = "protein"
        sequence_lines = []
        
        for line in lines:
            line = line.strip()
            if line.startswith(">"):
                header = line[1:]
                parts = header.split()
                if parts:
                    first_part = parts[0]
                    if '|' in first_part:
                        pipe_parts = first_part.split('|')
                        name = pipe_parts[2] if len(pipe_parts) >= 3 else pipe_parts[-1]
                    else:
                        name = first_part
            elif line:
                sequence_lines.append(line)
        
        return cls("".join(sequence_lines), name)

print("✓ SequenceHandler defined")

### 2.2 Mutation Handler

In [ ]:
@dataclass
class Mutation:
    """Represents a single point mutation."""
    original: str      # Original amino acid
    position: int      # Position (1-indexed)
    mutant: str        # Mutant amino acid
    
    def __post_init__(self):
        if self.original not in VALID_AMINO_ACIDS:
            raise ValueError(f"Invalid original amino acid: {self.original}")
        if self.mutant not in VALID_AMINO_ACIDS:
            raise ValueError(f"Invalid mutant amino acid: {self.mutant}")
        if self.position < 1:
            raise ValueError(f"Position must be >= 1")
    
    def __str__(self) -> str:
        return f"{self.original}{self.position}{self.mutant}"
    
    @classmethod
    def from_string(cls, mutation_str: str) -> "Mutation":
        """Parse mutation from string (e.g., 'A123G')."""
        mutation_str = mutation_str.strip().upper()
        pattern = r'^([A-Z])(\d+)([A-Z])$'
        match = re.match(pattern, mutation_str)
        
        if not match:
            raise ValueError(
                f"Invalid mutation format: '{mutation_str}'\n"
                f"Expected: [OriginalAA][Position][MutantAA] (e.g., A123G)"
            )
        
        original, position, mutant = match.groups()
        return cls(original=original, position=int(position), mutant=mutant)


class MutationHandler:
    """Handles mutation operations on sequences."""
    
    def __init__(self, wt_sequence: SequenceHandler):
        self.wt_sequence = wt_sequence
        self.mutations: List[Mutation] = []
    
    def add_mutation(self, mutation_str: str) -> Mutation:
        """Parse, validate, and add a mutation."""
        mutation = Mutation.from_string(mutation_str)
        
        # Validate position
        if mutation.position > len(self.wt_sequence):
            raise ValueError(
                f"Position {mutation.position} exceeds sequence length {len(self.wt_sequence)}"
            )
        
        # Validate original residue
        wt_residue = self.wt_sequence.get_residue(mutation.position)
        if wt_residue != mutation.original:
            raise ValueError(
                f"Mutation specifies {mutation.original} at position {mutation.position}, "
                f"but WT has {wt_residue}"
            )
        
        self.mutations.append(mutation)
        return mutation
    
    def get_mutant_sequence(self, mutation: Mutation) -> str:
        """Generate mutant sequence."""
        seq_list = list(str(self.wt_sequence))
        seq_list[mutation.position - 1] = mutation.mutant
        return "".join(seq_list)
    
    def get_mutant_handler(self, mutation: Mutation) -> SequenceHandler:
        """Get SequenceHandler for mutant."""
        mutant_seq = self.get_mutant_sequence(mutation)
        return SequenceHandler(mutant_seq, f"{self.wt_sequence.name}_{mutation}")

print("✓ MutationHandler defined")

### 2.3 BioEmu Runner

In [ ]:
@dataclass
class BioEmuConfig:
    """Configuration for BioEmu runs."""
    num_conformations: int = 100
    device: str = "cuda"  # or "cpu"
    seed: int = 42
    temperature: float = 1.0


class BioEmuRunner:
    """Runner for BioEmu conformational sampling."""
    
    def __init__(self, config: Optional[BioEmuConfig] = None):
        self.config = config or BioEmuConfig()
        self.bioemu_available = self._check_bioemu()
    
    def _check_bioemu(self) -> bool:
        """Check if BioEmu is available."""
        try:
            import bioemu
            return True
        except ImportError:
            return False
    
    def generate_conformations(
        self,
        sequence: str,
        output_dir: Path,
        name: str = "protein"
    ) -> Dict[str, Any]:
        """Generate conformational ensemble."""
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Save input
        input_data = {
            "sequence": sequence,
            "num_samples": self.config.num_conformations,
            "temperature": self.config.temperature,
            "seed": self.config.seed,
        }
        
        with open(output_dir / f"{name}_input.json", 'w') as f:
            json.dump(input_data, f, indent=2)
        
        # Save FASTA
        with open(output_dir / f"{name}_sequence.fasta", 'w') as f:
            f.write(f">{name}\n{sequence}\n")
        
        output_files = []
        
        if self.bioemu_available:
            try:
                import bioemu
                import torch
                
                print(f"  Loading BioEmu model on {self.config.device}...")
                torch.manual_seed(self.config.seed)
                
                # Initialize and run BioEmu
                sampler = bioemu.get_sampler(device=self.config.device)
                print(f"  Generating {self.config.num_conformations} conformations...")
                
                samples = sampler.sample(
                    sequence,
                    num_samples=self.config.num_conformations,
                    temperature=self.config.temperature
                )
                
                for i, sample in enumerate(samples):
                    path = output_dir / f"{name}_conf_{i:04d}.pdb"
                    if hasattr(sample, 'to_pdb'):
                        sample.to_pdb(str(path))
                    elif hasattr(sample, 'save'):
                        sample.save(str(path))
                    output_files.append(str(path))
                
                print(f"  ✓ Generated {len(output_files)} PDB files")
                
            except Exception as e:
                print(f"  ⚠ BioEmu error: {e}")
                print(f"  Creating placeholders for testing...")
                for i in range(self.config.num_conformations):
                    output_files.append(str(output_dir / f"{name}_conf_{i:04d}.pdb"))
        else:
            print(f"  ⚠ BioEmu not installed. Creating placeholders...")
            print(f"  Sequence length: {len(sequence)} residues")
            print(f"  Would generate {self.config.num_conformations} conformations")
            for i in range(self.config.num_conformations):
                output_files.append(str(output_dir / f"{name}_conf_{i:04d}.pdb"))
        
        result = {
            "name": name,
            "sequence_length": len(sequence),
            "num_conformations": self.config.num_conformations,
            "output_directory": str(output_dir),
            "output_files": output_files,
            "bioemu_used": self.bioemu_available,
        }
        
        with open(output_dir / f"{name}_metadata.json", 'w') as f:
            json.dump(result, f, indent=2)
        
        return result

print("✓ BioEmuRunner defined")
print(f"  BioEmu available: {BIOEMU_AVAILABLE}")

### 2.4 Alignment Module

In [ ]:
class StructureAligner:
    """Aligns protein structures using Kabsch algorithm."""
    
    @staticmethod
    def kabsch_rmsd(P: np.ndarray, Q: np.ndarray) -> Tuple[float, np.ndarray, np.ndarray]:
        """Calculate RMSD using Kabsch algorithm."""
        # Center structures
        centroid_P = np.mean(P, axis=0)
        centroid_Q = np.mean(Q, axis=0)
        
        P_centered = P - centroid_P
        Q_centered = Q - centroid_Q
        
        # SVD
        H = P_centered.T @ Q_centered
        U, S, Vt = np.linalg.svd(H)
        
        # Rotation matrix
        R = Vt.T @ U.T
        if np.linalg.det(R) < 0:
            Vt[-1, :] *= -1
            R = Vt.T @ U.T
        
        # RMSD
        P_aligned = P_centered @ R
        rmsd = np.sqrt(np.mean(np.sum((P_aligned - Q_centered) ** 2, axis=1)))
        
        return rmsd, R, centroid_Q - centroid_P @ R
    
    @staticmethod
    def align_trajectory(trajectory: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Align all frames to first frame."""
        n_frames = trajectory.shape[0]
        reference = trajectory[0]
        
        aligned = np.zeros_like(trajectory)
        rmsds = np.zeros(n_frames)
        
        for i in range(n_frames):
            rmsd, R, t = StructureAligner.kabsch_rmsd(trajectory[i], reference)
            aligned[i] = trajectory[i] @ R + t
            rmsds[i] = rmsd
        
        return aligned, rmsds
    
    @staticmethod
    def compute_rmsf(trajectory: np.ndarray) -> np.ndarray:
        """Compute per-residue RMSF."""
        average = np.mean(trajectory, axis=0)
        deviations = trajectory - average
        rmsf = np.sqrt(np.mean(np.sum(deviations ** 2, axis=2), axis=0))
        return rmsf

print("✓ StructureAligner defined")

### 2.5 Analysis Module

In [ ]:
class ConformationalAnalyzer:
    """Comprehensive conformational analysis."""
    
    @staticmethod
    def compute_radius_of_gyration(coords: np.ndarray) -> float:
        """Compute radius of gyration."""
        centroid = np.mean(coords, axis=0)
        distances = np.sqrt(np.sum((coords - centroid) ** 2, axis=1))
        return np.sqrt(np.mean(distances ** 2))
    
    @staticmethod
    def compute_rg_trajectory(trajectory: np.ndarray) -> np.ndarray:
        """Compute Rg for each frame."""
        return np.array([ConformationalAnalyzer.compute_radius_of_gyration(f) for f in trajectory])
    
    @staticmethod
    def compute_contact_map(coords: np.ndarray, cutoff: float = 8.0) -> np.ndarray:
        """Compute residue contact map."""
        n_residues = coords.shape[0]
        distances = np.zeros((n_residues, n_residues))
        
        for i in range(n_residues):
            for j in range(i, n_residues):
                dist = np.linalg.norm(coords[i] - coords[j])
                distances[i, j] = dist
                distances[j, i] = dist
        
        return (distances < cutoff).astype(float)
    
    @staticmethod
    def perform_pca(trajectory: np.ndarray, n_components: int = 10) -> Dict:
        """Perform PCA on trajectory."""
        n_frames = trajectory.shape[0]
        flattened = trajectory.reshape(n_frames, -1)
        
        # Center
        mean = np.mean(flattened, axis=0)
        centered = flattened - mean
        
        # Covariance and eigendecomposition
        cov = np.cov(centered.T)
        eigenvalues, eigenvectors = np.linalg.eigh(cov)
        
        # Sort descending
        idx = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idx][:n_components]
        eigenvectors = eigenvectors[:, idx][:, :n_components]
        
        # Project
        projections = centered @ eigenvectors
        
        # Variance explained
        total_var = np.sum(eigenvalues)
        explained = eigenvalues / total_var
        
        return {
            "eigenvalues": eigenvalues,
            "projections": projections,
            "explained_variance": explained,
            "cumulative_variance": np.cumsum(explained),
        }
    
    @staticmethod
    def compute_free_energy_landscape(
        projections: np.ndarray,
        bins: int = 50,
        temperature: float = 300.0
    ) -> Dict:
        """Compute 2D free energy landscape."""
        kB = 0.001987  # kcal/(mol·K)
        
        x = projections[:, 0]
        y = projections[:, 1]
        
        hist, xedges, yedges = np.histogram2d(x, y, bins=bins)
        prob = hist / np.sum(hist)
        prob[prob == 0] = 1e-10
        
        free_energy = -kB * temperature * np.log(prob)
        free_energy -= np.min(free_energy)
        
        return {
            "free_energy": free_energy,
            "x_centers": (xedges[:-1] + xedges[1:]) / 2,
            "y_centers": (yedges[:-1] + yedges[1:]) / 2,
        }

print("✓ ConformationalAnalyzer defined")

---
## 3. Interactive Pipeline

**⚠️ IMPORTANT: Run cells in order from top to bottom!**

The cells below depend on each other. Make sure you:
1. First run all cells in Section 1 (Setup) and Section 2 (Core Modules)
2. Then run the Input cells in order
3. Finally run the BioEmu generation and analysis cells

### 3.1 Input Wild-Type Sequence

### 3.0 Load Previous Analysis (Optional)

Run this cell to load and continue from a previous analysis saved in Google Drive.

In [ ]:
#@title Load Previous Analysis { display-mode: "form" }

# List previous analyses
previous_analyses = persistence.list_previous_analyses()

if not previous_analyses:
    print("📋 No previous analyses found.")
    print("   Continue with the cells below to start a new analysis.")
else:
    print("="*60)
    print("      PREVIOUS ANALYSES")
    print("="*60)
    print()
    
    # Create selection widget
    options = ["-- Select to load --"] + [
        f"{a['protein']} - {a['mutation']} ({a['timestamp'][:10]})"
        for a in previous_analyses
    ]
    
    for i, analysis in enumerate(previous_analyses, 1):
        print(f"  [{i}] {analysis['protein']} - {analysis['mutation']}")
        print(f"      📅 {analysis['timestamp'][:19]}")
        print(f"      📂 {analysis['path']}")
        print()
    
    #@markdown **Select analysis to load (0 = skip, start new):**
    load_index = 0  #@param {type:"integer"}
    
    if load_index > 0 and load_index <= len(previous_analyses):
        selected = previous_analyses[load_index - 1]
        print(f"\n📥 Loading: {selected['protein']} - {selected['mutation']}")
        
        # Load the analysis
        mutation_dir = Path(selected['path'])
        
        # Load summary
        summary_file = mutation_dir / "analysis_summary.json"
        if summary_file.exists():
            with open(summary_file) as f:
                loaded_summary = json.load(f)
            print(f"   ✓ Summary loaded")
        
        # Load sequence info
        wt_seq = loaded_summary.get('wild_type', {})
        print(f"   Protein: {wt_seq.get('name', 'unknown')}")
        print(f"   Length: {wt_seq.get('length', 'unknown')} residues")
        print(f"   Mutation: {loaded_summary.get('mutation', 'unknown')}")
        
        # Load trajectory data if exists
        trajectory_data = persistence.load_numpy_data(mutation_dir)
        if trajectory_data:
            wt_trajectory = trajectory_data.get('wt_trajectory')
            mutant_trajectory = trajectory_data.get('mutant_trajectory')
            print(f"   ✓ Trajectory data loaded")
        
        print("\n✓ Analysis loaded! You can now run the analysis cells.")
        print("  Skip to Section 4 (Analysis) to visualize the results.")
    else:
        print("\n➡️ Starting new analysis. Continue with the cells below.")

In [ ]:
# Use PersistenceManager for output directory
OUTPUT_BASE = persistence.output_dir

print("="*60)
print("         WILD-TYPE SEQUENCE INPUT")
print("="*60)
print()
print(f"📂 Results will be saved to: {OUTPUT_BASE}")
if DRIVE_MOUNTED:
    print("   ☁️ Google Drive sync enabled - results persist across sessions!")
print()
print("Enter your wild-type protein sequence below.")
print("Use standard one-letter amino acid codes (A, C, D, E, F, G, H, I, K, L, M, N, P, Q, R, S, T, V, W, Y)")
print()

In [ ]:
#@title Enter Wild-Type Sequence { display-mode: "form" }

#@markdown **Protein Name:**
protein_name = "AnnexinA11"  #@param {type:"string"}

#@markdown **Amino Acid Sequence:**
wt_sequence_input = "MSTVHEILCKLSLEGDHSTPPSAYGSVKAYTNFDAERDALNIETAIKTKGVDEVTIVNILTNRSNAQRQDIAFAYQRRTKKELASALKSALSGHLETVILGLLKTPAQYDASELKASMKGLGTDEDSLIEIICSRTNQELQEINRVYKEMYKTDLEKDIISDTSGDFRKLMVALAKGRRAEDGSVIDYELIDQDARDLYDAGVKRKGTDVPKWISIMTERSVPHLQKVFDRYKSYSPYDMLESIRKEVKGDLENAFLNLVQCIQNKPLYFADRLYDSMKGKGTRDKVLIRIMVSRSEVDMLKIRSEFKRKYGKSLYYYIQQDTKGDYQKALLYLCGGDD"  #@param {type:"string"}

# Validate and create handler
try:
    wt_handler = SequenceHandler(wt_sequence_input, protein_name)
    stats = wt_handler.get_stats()
    
    print("✓ Sequence validated successfully!")
    print()
    print(f"  Name: {wt_handler.name}")
    print(f"  Length: {stats['length']} amino acids")
    print(f"  Charged residues: {stats['charged']}")
    print(f"  Hydrophobic residues: {stats['hydrophobic']}")
    print(f"  Polar residues: {stats['polar']}")
    print()
    print(f"  First 50 residues: {str(wt_handler)[:50]}...")
    
except ValueError as e:
    print(f"✗ Error: {e}")

### 3.2 Input Mutation

In [ ]:
print("="*60)
print("            MUTATION INPUT")
print("="*60)
print()
print("Format: [OriginalAA][Position][MutantAA]")
print("Examples:")
print("  A123G  → Alanine at position 123 to Glycine")
print("  V456L  → Valine at position 456 to Leucine")
print()
print(f"Your sequence has {len(wt_handler)} residues (positions 1-{len(wt_handler)})")
print()

In [ ]:
#@title Enter Mutation { display-mode: "form" }

# Check if previous cell was executed
try:
    _ = wt_handler
except NameError:
    print("❌ ERROR: Run the 'Enter Wild-Type Sequence' cell first!")
    raise SystemExit("Run previous cell first")

#@markdown **Mutation (e.g., A123G):**
mutation_input = "G175E"  #@param {type:"string"}

# Validate mutation
try:
    mutation_handler = MutationHandler(wt_handler)
    mutation = mutation_handler.add_mutation(mutation_input)
    
    # Get mutant sequence
    mutant_handler = mutation_handler.get_mutant_handler(mutation)
    
    print("✓ Mutation validated successfully!")
    print()
    print(f"  Mutation: {mutation}")
    print(f"  Original residue: {mutation.original} at position {mutation.position}")
    print(f"  Mutant residue: {mutation.mutant}")
    print()
    
    # Show context
    start = max(0, mutation.position - 6)
    end = min(len(wt_handler), mutation.position + 5)
    wt_context = str(wt_handler)[start:end]
    mut_context = str(mutant_handler)[start:end]
    
    rel_pos = mutation.position - start - 1
    print(f"  Local context:")
    print(f"    WT:     ...{wt_context[:rel_pos]}[{wt_context[rel_pos]}]{wt_context[rel_pos+1:]}...")
    print(f"    Mutant: ...{mut_context[:rel_pos]}[{mut_context[rel_pos]}]{mut_context[rel_pos+1:]}...")
    
except NameError as e:
    print(f"❌ ERROR: {e}")
    print("   Make sure you ran the Core Modules cells (Section 2)")
except ValueError as e:
    print(f"❌ Error: {e}")

### 3.3 Configure BioEmu

In [ ]:
#@title BioEmu Configuration { display-mode: "form" }

#@markdown **Number of conformations to generate:**
num_conformations = 100  #@param {type:"integer"}

#@markdown **Device:**
device = "cuda"  #@param ["cuda", "cpu"]

#@markdown **Random seed:**
seed = 42  #@param {type:"integer"}

#@markdown **Sampling temperature:**
temperature = 1.0  #@param {type:"number"}

# Create config
bioemu_config = BioEmuConfig(
    num_conformations=num_conformations,
    device=device,
    seed=seed,
    temperature=temperature
)

print("Configuration:")
print(f"  • Conformations: {bioemu_config.num_conformations}")
print(f"  • Device: {bioemu_config.device}")
print(f"  • Seed: {bioemu_config.seed}")
print(f"  • Temperature: {bioemu_config.temperature}")

### 3.4 Run BioEmu Conformation Generation

In [ ]:
# Check if previous cells were executed
_missing_vars = []
for var_name in ['wt_handler', 'mutation', 'mutant_handler', 'bioemu_config', 'OUTPUT_BASE', 'persistence', 'DRIVE_MOUNTED']:
    if var_name not in dir():
        _missing_vars.append(var_name)

if _missing_vars:
    print("❌ ERROR: Please run all previous cells first!")
    print(f"   Missing variables: {', '.join(_missing_vars)}")
    print()
    print("   Run cells in order:")
    print("   1. Setup and Installation (Cell 3)")
    print("   2. Google Drive Persistence (Cells 5-6)")
    print("   3. Import Libraries (Cell 7)")
    print("   4. Core Modules - SequenceHandler (Cell 10)")
    print("   5. Core Modules - MutationHandler (Cell 12)")
    print("   6. Core Modules - BioEmuRunner (Cell 14)")
    print("   7. Input Wild-Type Sequence (Cells 23-24)")
    print("   8. Input Mutation (Cells 26-27)")
    print("   9. Configure BioEmu (Cell 29)")
    print("   10. Then run this cell")
    raise ValueError("Run previous cells first - see instructions above")

print("="*60)
print("      BIOEMU CONFORMATION GENERATION")
print("="*60)
print()

# Create output directories using persistence manager
mutation_dir = persistence.get_output_path(wt_handler.name, str(mutation))
wt_dir = mutation_dir / "wt"
mutant_dir = mutation_dir / "mutant"

print(f"Output directory: {mutation_dir}")
if DRIVE_MOUNTED:
    print("☁️ Results will be automatically saved to Google Drive")
print()

# Check for existing checkpoint
checkpoint = persistence.load_checkpoint("generation", mutation_dir)
use_existing = 'n'

if checkpoint:
    print("📥 Found existing generation checkpoint!")
    print(f"   Generated: {checkpoint.get('timestamp', 'unknown')}")
    print()
    use_existing = input("Use existing results? (y/n): ").strip().lower()
    if use_existing == 'y':
        wt_result = checkpoint.get('wt_result', {})
        mutant_result = checkpoint.get('mutant_result', {})
        print("✓ Loaded existing results")
        print("="*60)
        print("  GENERATION LOADED FROM CHECKPOINT")
        print("="*60)

if not checkpoint or use_existing != 'y':
    # Initialize runner
    runner = BioEmuRunner(bioemu_config)

    # Generate WT conformations
    print("[1/2] Generating Wild-Type conformations...")
    wt_result = runner.generate_conformations(
        str(wt_handler),
        wt_dir,
        f"{wt_handler.name}_WT"
    )
    print(f"  ✓ Complete: {wt_dir}")
    print()

    # Generate Mutant conformations
    print("[2/2] Generating Mutant conformations...")
    mutant_result = runner.generate_conformations(
        str(mutant_handler),
        mutant_dir,
        f"{mutant_handler.name}"
    )
    print(f"  ✓ Complete: {mutant_dir}")
    print()
    
    # Save checkpoint
    persistence.save_checkpoint({
        "timestamp": datetime.now().isoformat(),
        "wt_result": wt_result,
        "mutant_result": mutant_result,
        "config": {
            "num_conformations": bioemu_config.num_conformations,
            "device": bioemu_config.device,
            "seed": bioemu_config.seed,
            "temperature": bioemu_config.temperature
        }
    }, "generation", mutation_dir)

    print("="*60)
    print("  GENERATION COMPLETE")
    print("="*60)

---
## 4. Analysis (Demo with Simulated Data)

The following cells demonstrate the analysis workflow using simulated conformational data.

Once you have real BioEmu outputs, replace the simulated data with loaded trajectories.

### 4.1 Generate Simulated Data for Demo

In [ ]:
# Check for existing trajectory data
existing_data = persistence.load_numpy_data(mutation_dir)

if existing_data is not None:
    print("📥 Found existing trajectory data!")
    wt_trajectory = existing_data['wt_trajectory']
    mutant_trajectory = existing_data['mutant_trajectory']
    n_frames = wt_trajectory.shape[0]
    n_residues = wt_trajectory.shape[1]
    print(f"  WT trajectory: {wt_trajectory.shape}")
    print(f"  Mutant trajectory: {mutant_trajectory.shape}")
    print("✓ Loaded trajectories from checkpoint")
else:
    # Simulate conformational ensembles for demonstration
    np.random.seed(42)

    n_frames = 100
    n_residues = len(wt_handler)

    # Simulate WT trajectory (CA coordinates)
    # Start with a baseline structure and add fluctuations
    baseline_wt = np.zeros((n_residues, 3))
    for i in range(n_residues):
        baseline_wt[i] = [i * 3.8, np.sin(i * 0.1) * 5, np.cos(i * 0.1) * 5]  # Extended chain

    # Add thermal fluctuations
    wt_trajectory = np.array([
        baseline_wt + np.random.randn(n_residues, 3) * 1.5
        for _ in range(n_frames)
    ])

    # Simulate Mutant trajectory (slightly different dynamics at mutation site)
    mut_baseline = baseline_wt.copy()
    mut_idx = mutation.position - 1

    # Add local perturbation at mutation site
    mutant_trajectory = np.array([
        mut_baseline + np.random.randn(n_residues, 3) * 1.5
        for _ in range(n_frames)
    ])

    # Increase flexibility around mutation site
    for i in range(max(0, mut_idx-5), min(n_residues, mut_idx+6)):
        mutant_trajectory[:, i, :] += np.random.randn(n_frames, 3) * 0.8

    # Save trajectory data for persistence
    persistence.save_numpy_data({
        'wt_trajectory': wt_trajectory,
        'mutant_trajectory': mutant_trajectory
    }, mutation_dir)

    print(f"✓ Simulated trajectories created and saved")
    print(f"  WT trajectory: {wt_trajectory.shape}")
    print(f"  Mutant trajectory: {mutant_trajectory.shape}")

### 4.2 Align Structures

In [ ]:
print("Aligning trajectories...")

# Align WT
wt_aligned, wt_rmsds = StructureAligner.align_trajectory(wt_trajectory)
print(f"  WT RMSD: {wt_rmsds.mean():.2f} ± {wt_rmsds.std():.2f} Å")

# Align Mutant
mut_aligned, mut_rmsds = StructureAligner.align_trajectory(mutant_trajectory)
print(f"  Mutant RMSD: {mut_rmsds.mean():.2f} ± {mut_rmsds.std():.2f} Å")

# Compute RMSF
wt_rmsf = StructureAligner.compute_rmsf(wt_aligned)
mut_rmsf = StructureAligner.compute_rmsf(mut_aligned)

print("\n✓ Alignment complete")

### 4.3 RMSF Analysis

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot RMSF
residue_numbers = np.arange(1, n_residues + 1)

axes[0].plot(residue_numbers, wt_rmsf, 'b-', label='Wild-Type', alpha=0.8)
axes[0].plot(residue_numbers, mut_rmsf, 'r-', label='Mutant', alpha=0.8)
axes[0].axvline(mutation.position, color='green', linestyle='--', label=f'Mutation ({mutation})')
axes[0].fill_between(residue_numbers, wt_rmsf, mut_rmsf, 
                      where=mut_rmsf > wt_rmsf, alpha=0.3, color='red')
axes[0].fill_between(residue_numbers, wt_rmsf, mut_rmsf, 
                      where=mut_rmsf <= wt_rmsf, alpha=0.3, color='blue')
axes[0].set_xlabel('Residue Number')
axes[0].set_ylabel('RMSF (Å)')
axes[0].set_title('Per-Residue Flexibility (RMSF)')
axes[0].legend()
axes[0].set_xlim(1, n_residues)

# Plot ΔRMSF
delta_rmsf = mut_rmsf - wt_rmsf
colors = ['red' if d > 0 else 'blue' for d in delta_rmsf]
axes[1].bar(residue_numbers, delta_rmsf, color=colors, alpha=0.7)
axes[1].axhline(0, color='black', linewidth=0.5)
axes[1].axvline(mutation.position, color='green', linestyle='--')
axes[1].set_xlabel('Residue Number')
axes[1].set_ylabel('ΔRMSF (Mutant - WT) (Å)')
axes[1].set_title('Change in Flexibility Upon Mutation')
axes[1].set_xlim(1, n_residues)

plt.tight_layout()
plt.savefig(mutation_dir / 'rmsf_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nΔRMSF at mutation site (residue {mutation.position}): {delta_rmsf[mutation.position-1]:+.2f} Å")

### 4.4 Radius of Gyration Analysis

In [ ]:
# Compute Rg
wt_rg = ConformationalAnalyzer.compute_rg_trajectory(wt_aligned)
mut_rg = ConformationalAnalyzer.compute_rg_trajectory(mut_aligned)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution plot
axes[0].hist(wt_rg, bins=30, alpha=0.6, label='Wild-Type', color='blue', edgecolor='black')
axes[0].hist(mut_rg, bins=30, alpha=0.6, label='Mutant', color='red', edgecolor='black')
axes[0].axvline(wt_rg.mean(), color='blue', linestyle='--', linewidth=2)
axes[0].axvline(mut_rg.mean(), color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Radius of Gyration (Å)')
axes[0].set_ylabel('Count')
axes[0].set_title('Radius of Gyration Distribution')
axes[0].legend()

# Box plot
data = [wt_rg, mut_rg]
bp = axes[1].boxplot(data, labels=['Wild-Type', 'Mutant'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
axes[1].set_ylabel('Radius of Gyration (Å)')
axes[1].set_title('Rg Comparison')

plt.tight_layout()
plt.savefig(mutation_dir / 'rg_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nRadius of Gyration:")
print(f"  WT:     {wt_rg.mean():.2f} ± {wt_rg.std():.2f} Å")
print(f"  Mutant: {mut_rg.mean():.2f} ± {mut_rg.std():.2f} Å")
print(f"  ΔRg:    {mut_rg.mean() - wt_rg.mean():+.2f} Å")

### 4.5 PCA Analysis

In [ ]:
# Combine trajectories for joint PCA
combined = np.vstack([wt_aligned, mut_aligned])

# Perform PCA
pca_results = ConformationalAnalyzer.perform_pca(combined, n_components=10)

# Split projections
wt_proj = pca_results['projections'][:n_frames]
mut_proj = pca_results['projections'][n_frames:]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# PC1 vs PC2
axes[0].scatter(wt_proj[:, 0], wt_proj[:, 1], alpha=0.6, c='blue', label='Wild-Type', s=30)
axes[0].scatter(mut_proj[:, 0], mut_proj[:, 1], alpha=0.6, c='red', label='Mutant', s=30)
axes[0].set_xlabel(f"PC1 ({pca_results['explained_variance'][0]*100:.1f}%)")
axes[0].set_ylabel(f"PC2 ({pca_results['explained_variance'][1]*100:.1f}%)")
axes[0].set_title('Conformational Space (PCA)')
axes[0].legend()

# Variance explained
pcs = range(1, 11)
axes[1].bar(pcs, pca_results['explained_variance'] * 100, alpha=0.7, label='Individual')
axes[1].plot(pcs, pca_results['cumulative_variance'] * 100, 'ro-', label='Cumulative')
axes[1].set_xlabel('Principal Component')
axes[1].set_ylabel('Variance Explained (%)')
axes[1].set_title('PCA Variance')
axes[1].legend()
axes[1].set_xticks(pcs)

plt.tight_layout()
plt.savefig(mutation_dir / 'pca_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFirst 3 PCs explain {pca_results['cumulative_variance'][2]*100:.1f}% of variance")

### 4.6 Free Energy Landscape

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# WT Free Energy
wt_fel = ConformationalAnalyzer.compute_free_energy_landscape(wt_proj)
im1 = axes[0].contourf(wt_fel['x_centers'], wt_fel['y_centers'], 
                        wt_fel['free_energy'].T, levels=20, cmap='RdYlBu_r')
axes[0].scatter(wt_proj[:, 0], wt_proj[:, 1], c='black', s=5, alpha=0.3)
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].set_title('Wild-Type Free Energy')
plt.colorbar(im1, ax=axes[0], label='ΔG (kcal/mol)')

# Mutant Free Energy
mut_fel = ConformationalAnalyzer.compute_free_energy_landscape(mut_proj)
im2 = axes[1].contourf(mut_fel['x_centers'], mut_fel['y_centers'], 
                        mut_fel['free_energy'].T, levels=20, cmap='RdYlBu_r')
axes[1].scatter(mut_proj[:, 0], mut_proj[:, 1], c='black', s=5, alpha=0.3)
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')
axes[1].set_title('Mutant Free Energy')
plt.colorbar(im2, ax=axes[1], label='ΔG (kcal/mol)')

# Overlay comparison
axes[2].scatter(wt_proj[:, 0], wt_proj[:, 1], alpha=0.5, c='blue', label='WT', s=20)
axes[2].scatter(mut_proj[:, 0], mut_proj[:, 1], alpha=0.5, c='red', label='Mutant', s=20)

# Add density contours
from scipy.stats import gaussian_kde
wt_xy = np.vstack([wt_proj[:, 0], wt_proj[:, 1]])
mut_xy = np.vstack([mut_proj[:, 0], mut_proj[:, 1]])

axes[2].set_xlabel('PC1')
axes[2].set_ylabel('PC2')
axes[2].set_title('Conformational Space Overlap')
axes[2].legend()

plt.tight_layout()
plt.savefig(mutation_dir / 'free_energy_landscape.png', dpi=150, bbox_inches='tight')
plt.show()

### 4.7 Contact Map Analysis

In [ ]:
# Compute average contact maps
print("Computing contact maps (this may take a moment)...")

# Sample frames for efficiency
sample_frames = min(20, n_frames)
indices = np.linspace(0, n_frames-1, sample_frames, dtype=int)

wt_contacts = np.mean([ConformationalAnalyzer.compute_contact_map(wt_aligned[i]) for i in indices], axis=0)
mut_contacts = np.mean([ConformationalAnalyzer.compute_contact_map(mut_aligned[i]) for i in indices], axis=0)

# Differential contact map
diff_contacts = mut_contacts - wt_contacts

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# WT contacts
im1 = axes[0].imshow(wt_contacts, cmap='YlOrRd', aspect='auto')
axes[0].set_xlabel('Residue')
axes[0].set_ylabel('Residue')
axes[0].set_title('WT Contact Probability')
plt.colorbar(im1, ax=axes[0])

# Mutant contacts
im2 = axes[1].imshow(mut_contacts, cmap='YlOrRd', aspect='auto')
axes[1].set_xlabel('Residue')
axes[1].set_ylabel('Residue')
axes[1].set_title('Mutant Contact Probability')
plt.colorbar(im2, ax=axes[1])

# Differential
vmax = np.abs(diff_contacts).max()
im3 = axes[2].imshow(diff_contacts, cmap='RdBu_r', aspect='auto', vmin=-vmax, vmax=vmax)
axes[2].axhline(mutation.position-1, color='green', linewidth=0.5)
axes[2].axvline(mutation.position-1, color='green', linewidth=0.5)
axes[2].set_xlabel('Residue')
axes[2].set_ylabel('Residue')
axes[2].set_title('Differential (Mutant - WT)')
plt.colorbar(im3, ax=axes[2], label='ΔContact')

plt.tight_layout()
plt.savefig(mutation_dir / 'contact_maps.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Contact map analysis complete")

---
## 5. Summary Report

In [ ]:
# Generate summary
summary = {
    "timestamp": datetime.now().isoformat(),
    "wild_type": {
        "name": wt_handler.name,
        "length": len(wt_handler),
        "sequence": str(wt_handler)[:100] + "..." if len(wt_handler) > 100 else str(wt_handler)
    },
    "mutation": str(mutation),
    "mutation_details": {
        "original": mutation.original,
        "position": mutation.position,
        "mutant": mutation.mutant
    },
    "analysis": {
        "rmsd": {
            "wt_mean": float(wt_rmsds.mean()),
            "wt_std": float(wt_rmsds.std()),
            "mutant_mean": float(mut_rmsds.mean()),
            "mutant_std": float(mut_rmsds.std()),
        },
        "rg": {
            "wt_mean": float(wt_rg.mean()),
            "wt_std": float(wt_rg.std()),
            "mutant_mean": float(mut_rg.mean()),
            "mutant_std": float(mut_rg.std()),
            "delta": float(mut_rg.mean() - wt_rg.mean()),
        },
        "rmsf_at_mutation": {
            "wt": float(wt_rmsf[mutation.position-1]),
            "mutant": float(mut_rmsf[mutation.position-1]),
            "delta": float(delta_rmsf[mutation.position-1]),
        },
        "pca_variance_explained_3pc": float(pca_results['cumulative_variance'][2]),
    },
    "persistence": {
        "drive_mounted": DRIVE_MOUNTED,
        "output_directory": str(mutation_dir)
    }
}

# Save summary using persistence manager
with open(mutation_dir / 'analysis_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

# Save analysis checkpoint for easy reload
persistence.save_checkpoint({
    "summary": summary,
    "analysis_complete": True
}, "analysis", mutation_dir)

# Display summary
print("="*60)
print("              ANALYSIS SUMMARY")
print("="*60)
print()
print(f"  Protein: {wt_handler.name}")
print(f"  Mutation: {mutation}")
print(f"  Sequence length: {len(wt_handler)} residues")
print()
print("  RMSD:")
print(f"    WT:     {summary['analysis']['rmsd']['wt_mean']:.2f} ± {summary['analysis']['rmsd']['wt_std']:.2f} Å")
print(f"    Mutant: {summary['analysis']['rmsd']['mutant_mean']:.2f} ± {summary['analysis']['rmsd']['mutant_std']:.2f} Å")
print()
print("  Radius of Gyration:")
print(f"    WT:     {summary['analysis']['rg']['wt_mean']:.2f} ± {summary['analysis']['rg']['wt_std']:.2f} Å")
print(f"    Mutant: {summary['analysis']['rg']['mutant_mean']:.2f} ± {summary['analysis']['rg']['mutant_std']:.2f} Å")
print(f"    ΔRg:    {summary['analysis']['rg']['delta']:+.2f} Å")
print()
print(f"  RMSF at mutation site (residue {mutation.position}):")
print(f"    WT:     {summary['analysis']['rmsf_at_mutation']['wt']:.2f} Å")
print(f"    Mutant: {summary['analysis']['rmsf_at_mutation']['mutant']:.2f} Å")
print(f"    ΔRMSF:  {summary['analysis']['rmsf_at_mutation']['delta']:+.2f} Å")
print()
print(f"  PCA: First 3 PCs explain {summary['analysis']['pca_variance_explained_3pc']*100:.1f}% of variance")
print()
print("="*60)
print(f"  Results saved to: {mutation_dir}")
if DRIVE_MOUNTED:
    print("  ☁️ Results synced to Google Drive - will persist across sessions!")
print("="*60)

---
## 6. Binder Design Considerations

Based on the analysis, consider the following for binder design:

In [ ]:
print("\n" + "="*60)
print("         BINDER DESIGN RECOMMENDATIONS")
print("="*60)
print()

# Analyze mutation effects
delta_rmsf_mutation = delta_rmsf[mutation.position-1]
delta_rg_val = summary['analysis']['rg']['delta']

print(f"Mutation {mutation} Analysis:")
print()

# Flexibility change
if delta_rmsf_mutation > 0.5:
    print(f"  ⚠ INCREASED FLEXIBILITY at mutation site (+{delta_rmsf_mutation:.2f} Å)")
    print("    → Consider binders that stabilize the local region")
    print("    → Target conformations where mutation site is accessible")
elif delta_rmsf_mutation < -0.5:
    print(f"  ⚠ DECREASED FLEXIBILITY at mutation site ({delta_rmsf_mutation:.2f} Å)")
    print("    → Mutation may rigidify the structure")
    print("    → Consider allosteric binding sites")
else:
    print(f"  ✓ Minimal flexibility change at mutation site ({delta_rmsf_mutation:+.2f} Å)")

print()

# Compactness change
if abs(delta_rg_val) > 1.0:
    if delta_rg_val > 0:
        print(f"  ⚠ EXPANSION detected (ΔRg = +{delta_rg_val:.2f} Å)")
        print("    → Mutation may cause partial unfolding")
        print("    → Consider binders that promote compact state")
    else:
        print(f"  ⚠ COMPACTION detected (ΔRg = {delta_rg_val:.2f} Å)")
        print("    → Mutation may cause over-stabilization")
else:
    print(f"  ✓ Minimal compactness change (ΔRg = {delta_rg_val:+.2f} Å)")

print()
print("Suggested Next Steps:")
print("  1. Identify mutation-specific conformations from clustering")
print("  2. Analyze surface accessibility at mutation site")
print("  3. Design binders targeting exposed regions")
print("  4. Test binder affinity using molecular docking")
print()

---
## 7. Download Results

In [ ]:
# Create a zip file of all results
import shutil

zip_path = OUTPUT_BASE / f"{wt_handler.name}_{mutation}_results"
shutil.make_archive(str(zip_path), 'zip', mutation_dir)

print(f"Results archived to: {zip_path}.zip")
print()

# Persistence status
if DRIVE_MOUNTED:
    print("="*60)
    print("   ☁️ GOOGLE DRIVE PERSISTENCE STATUS")
    print("="*60)
    print()
    print(f"✓ All results saved to: {mutation_dir}")
    print("✓ Results will persist across Colab sessions!")
    print()
    print("📁 Files saved:")
    for f in mutation_dir.rglob("*"):
        if f.is_file():
            rel_path = f.relative_to(mutation_dir)
            size_kb = f.stat().st_size / 1024
            print(f"   • {rel_path} ({size_kb:.1f} KB)")
    print()
    print("💡 To access in future sessions:")
    print("   1. Run the Setup and Persistence cells")
    print("   2. Your results will be automatically loaded!")
else:
    print("⚠ Google Drive not mounted - results saved locally")
    print("  Results will be lost when session ends!")

print()

# For Colab, provide download link
try:
    from google.colab import files
    print("📥 Download option:")
    files.download(f"{zip_path}.zip")
except:
    print("Download the results from the file browser on the left.")